In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

API = "https://data.cityofnewyork.us/resource/h9gi-nx95.csv"

# Get last 7 days
end = datetime.now().strftime('%Y-%m-%d')
start = (datetime.now() - timedelta(7)).strftime('%Y-%m-%d')

# Fetch data
params = {'$limit': 1000, '$where': f"crash_date between '{start}' and '{end}'"}
response = requests.get(API, params=params)

# Save
filename = f"data/collisions_{start}_to_{end}.csv"
open(filename, 'wb').write(response.content)

# Display
df = pd.read_csv(filename)
print(f"✅ {len(df)} records saved to {filename}")
print(df['borough'].value_counts())

✅ 728 records saved to data/collisions_2026-01-14_to_2026-01-21.csv
borough
BROOKLYN         198
QUEENS           154
MANHATTAN        113
BRONX             96
STATEN ISLAND     30
Name: count, dtype: int64
